# RAG System

![rag](../images/rag.png)

In [2]:
import tiktoken
from langchain_openai import OpenAIEmbeddings
import numpy as np
import bs4
from langchain_community.document_loaders import WebBaseLoader

import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

open_api_key = os.getenv("OPEN_API_KEY")

os.environ['OPENAI_API_KEY'] = open_api_key
os.environ['USER_AGENT'] = os.getenv("USER_AGENT")
USER_AGENT = os.getenv("USER_AGENT")

In [2]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

Count tokens

In [3]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [4]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)


1536

In [5]:
print(query_result[:20])

[-0.0036279945634305477, -0.0031491245608776808, -0.012412562035024166, -0.02501540258526802, -0.02831358090043068, 0.017581820487976074, -0.015539486892521381, -0.008543546311557293, -0.006773947738111019, -0.004887009970843792, -0.003742162138223648, 0.005448334384709597, -0.008372295647859573, -0.001477836980484426, 0.009444202296435833, 0.019548039883375168, 0.03077452816069126, 0.014968648552894592, 0.01575513742864132, -0.015501431189477444]


Cosine Similarity

In [6]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.880685727045271


Retrieval (documents in vector space) augmented generation (llm)

In [7]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://mathmod.chnu.edu.ua/",)
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
blog_docs = loader.load()

In [10]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)
print(splits[:2])

# splits = blog_docs.split(' ')
# print(splits)

[Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content='Кафедра математичного моделювання - Кафедра математичного моделювання\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nПерейти до основного вмісту\n\n\n\n\n\n\n[email\xa0protected]\n\n\n\r\n                 вул. Університетська 28, каб. 17\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНовини\n\nПро нас'), Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, pag

In [ ]:
# # Index
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
# vectorstore = Chroma.from_documents(documents=splits, 
#                                     embedding=OpenAIEmbeddings())

# retriever = vectorstore.as_retriever()

In [ ]:
# # Index
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
# vectorstore = Chroma.from_documents(documents=splits, 
#                                     embedding=OpenAIEmbeddings())


# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [11]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [12]:
docs = retriever.get_relevant_documents("Хто такий Ігор Михайлович Черевко?")
print(docs)

C:\Users\vmelnyk2\AppData\Local\Temp\ipykernel_28676\3043429598.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Хто такий Ігор Михайлович Черевко?")


[Document(id='1789da48-20bd-4531-82ac-36a6db3f4de2', metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content="Черевко Ігор Михайлович\nЗавідувач кафедри\n\n\n\n\nОстанні новини\n\n\n\n20лют\nВшанування пам'яті Героїв Небесної Сотні\n\n\n\n\n20лют\nУспішне міжнародне стажування викладачок кафедри математичного моделювання\n\n\n\n\n15лют\nПідвищення кваліфікації викладачів кафедри")]


In [13]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Хто такий Ігор Михайлович Черевко?")

'Ігор Михайлович Черевко - завідувач кафедри математичного моделювання.'

## RAG system all together

parse all links

In [3]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

with open('../resources/sets/rag-links.txt', 'r') as file:
    data = file.read().replace('\n', ',')

links = data.split(',')

links_no_pdf = [x for x in links if ".pdf" not in x]

loader = WebBaseLoader(
    web_paths=links_no_pdf
)
blog_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)
print(splits[:2])

vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())

vectorstore.save_local("../models/")

# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

[Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, page_content='Кафедра математичного моделювання - Кафедра математичного моделювання\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nПерейти до основного вмісту\n\n\n\n\n\n\n[email\xa0protected]\n\n\n\r\n                 вул. Університетська 28, каб. 17\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНовини\n\nПро нас'), Document(metadata={'source': 'https://mathmod.chnu.edu.ua/', 'title': 'Кафедра математичного моделювання - Кафедра математичного моделювання', 'description': 'Офіційна сторінка кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича', 'language': 'uk'}, pag

In [ ]:
# Now store pdf files in multiple models:

with open('../resources/sets/rag-links.txt', 'r') as file:
    data = file.read().replace('\n', ',')
links = data.split(',')
links_pdf = [x for x in links if ".pdf" in x]

# Load documents
loader = WebBaseLoader(web_paths=links_pdf)
blog_docs = loader.load()

# Split text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(blog_docs)

print(len(splits))

# Define chunk size for FAISS sharding
batch_size = 500  # Adjust based on memory constraints
num_batches = (len(splits) // batch_size) + 1

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Save FAISS indexes in multiple files
save_path = "../models/"
os.makedirs(save_path, exist_ok=True)

for i in range(len(splits)):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_splits = splits[start_idx:end_idx]

    if batch_splits:  # Avoid empty batches
        vectorstore = FAISS.from_documents(batch_splits, embeddings)
        vectorstore.save_local(os.path.join(save_path, f"faiss_index_{i}"))
        if i % 10 == 0:
            print(f"Saved FAISS shard {i} with {len(batch_splits)} documents")


106337
Saved FAISS shard 0 with 500 documents
Saved FAISS shard 1 with 500 documents
Saved FAISS shard 2 with 500 documents
Saved FAISS shard 3 with 500 documents
Saved FAISS shard 4 with 500 documents
Saved FAISS shard 5 with 500 documents
Saved FAISS shard 6 with 500 documents
Saved FAISS shard 7 with 500 documents
Saved FAISS shard 8 with 500 documents
Saved FAISS shard 9 with 500 documents
Saved FAISS shard 10 with 500 documents
Saved FAISS shard 11 with 500 documents
Saved FAISS shard 12 with 500 documents
Saved FAISS shard 13 with 500 documents
Saved FAISS shard 14 with 500 documents
Saved FAISS shard 15 with 500 documents
Saved FAISS shard 16 with 500 documents
Saved FAISS shard 17 with 500 documents
Saved FAISS shard 18 with 500 documents
Saved FAISS shard 19 with 500 documents
Saved FAISS shard 20 with 500 documents
Saved FAISS shard 21 with 500 documents
Saved FAISS shard 22 with 500 documents
Saved FAISS shard 23 with 500 documents
Saved FAISS shard 24 with 500 documents
Sav

In [8]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pickle

embd = OpenAIEmbeddings()

save_path = "../models/"
with open(f"../models/index.pkl", "rb") as f:
    stored_data = pickle.load(f)

vectorstore_init = FAISS.load_local(folder_path="../models/", embeddings=embd, allow_dangerous_deserialization=True)

# Load all FAISS shards and merge them
faiss_indexes = [vectorstore_init]
for filename in os.listdir(save_path):
    if filename.startswith("faiss_index_"):  # Load only the FAISS shards
        index_path = os.path.join(save_path, filename)
        vectorstore = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
        faiss_indexes.append(vectorstore)

# Merge all FAISS indexes into a single one
if faiss_indexes:
    merged_vectorstore = faiss_indexes[0]
    for store in faiss_indexes[1:]:
        merged_vectorstore.merge_from(store)

# retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # number of documents retrieved

retriever = merged_vectorstore.as_retriever(search_kwargs={"k": 5})

# Prompt
template = """Answer the question based only on the following context:
{context}

Please, provide as much information as possible!
Do not say words like 'context'
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
rag_chain.invoke("Хто такий Ігор Михайлович Черевко?")

'Ігор Михайлович Черевко є завідувачем кафедри математичного моделювання Чернівецького національного університету імені Юрія Федьковича. Він народився у 1956 році. Черевко має ступінь доктора фізико-математичних наук і є професором. Він також залучений до викладання курсів, таких як "Математичне моделювання динамічних систем і процесів" та "Комп\'ютерне моделювання жорстких процесів та систем".'

In [10]:
rag_chain.invoke("Розскажи про навчальну програму 'комп`ютерні науки'")

'Навчальна програма з комп\'ютерних наук на кафедрі математичного моделювання охоплює як бакалаврський, так і магістерський рівні вищої освіти. \n\nДля бакалаврів спеціальності 122 "Комп\'ютерні науки" пропонуються робочі програми обов\'язкових та вибіркових навчальних дисциплін. Цільова група для цих програм - студенти першого (бакалаврського) рівня вищої освіти.\n\nМагістри, які навчаються за тією ж спеціальністю, також мають доступ до робочих програм обов\'язкових та вибіркових дисциплін. Цільова група для цих програм - студенти другого (магістерського) рівня вищої освіти.\n\nКрім того, для магістрів другого курсу спеціальності 122 "Комп\'ютерні науки" проводиться проєктний практикум, що є частиною навчальної програми. Цей практикум спрямований на розвиток практичних навичок і застосування теоретичних знань у реальних проєктах.'

todo:
- deploy python code with vectorstore
- multiquery 